# COVIDvu

COVID-19 view various charts.

---
## Runtime prerequisites

In [ ]:
%%capture --no-stderr requirementsOutput

displayRequirementsOutput = False

%pip install -r requirements.txt
from utils import autoReloadCode; autoReloadCode()

In [ ]:
if displayRequirementsOutput:
    requirementsOutput.show()

---
## Pull latest datasets from the JH CSSE GitHub repository

This function is `crontab` ready; it can be called from here, the command line, or as a system job.

In [ ]:
%sx ./refreshdata

## Group confirmed cases by country

In [ ]:
import os

import pandas as pd
import numpy as np
import utils
pd.options.mode.chained_assignment = None

In [ ]:
COVID19_PATH=os.path.join(os.path.join(os.getcwd(), 'COVID-19'), 'csse_covid_19_data/csse_covid_19_time_series')
CONFIRMED_FILE=os.path.join(COVID19_PATH, 'time_series_19-covid-Confirmed.csv')

CONFIRMED_FILE

In [ ]:
DEATHS_FILE=os.path.join(COVID19_PATH, 'time_series_19-covid-Deaths.csv')
DEATHS_FILE

In [ ]:
RECOVERY_FILE=os.path.join(COVID19_PATH, 'time_series_19-covid-Recovered.csv')
RECOVERY_FILE

In [ ]:
confirmedCasesAll = pd.read_csv(CONFIRMED_FILE)
confirmedCasesUS = confirmedCasesAll[confirmedCasesAll['Country/Region']=='US'].drop(
    'Country/Region', axis=1).set_index('Province/State').T
confirmedCasesUS.head()

In [ ]:
confirmedCases = pd.read_csv(CONFIRMED_FILE).groupby(['Country/Region']).sum().T
confirmedCases.head()

In [ ]:
confirmedDeaths = pd.read_csv(DEATHS_FILE).groupby(['Country/Region']).sum().T
confirmedDeaths.head()

In [ ]:
confirmedRecovered = pd.read_csv(RECOVERY_FILE).groupby(['Country/Region']).sum().T

confirmedRecovered.head()

In [ ]:
confirmedCasesTrim = confirmedCases.iloc[2:]
confirmedCasesTrim.index = pd.to_datetime(confirmedCasesTrim.index)
confirmedCasesTrim = utils.computeGlobal(confirmedCasesTrim)
confirmedCasesTrim = utils.computeCasesOutside(confirmedCasesTrim, 
                                               ['Mainland China', '!Global'],
                                               '!Outside Mainland China'
                                              )

In [ ]:
confirmedCasesUSTrim = confirmedCasesUS.iloc[2:]
confirmedCasesUSTrim.index = pd.to_datetime(confirmedCasesUSTrim.index)
confirmedCasesUSTrim['!Total US'] = confirmedCasesUSTrim.sum(axis=1)
confirmedCasesUSTrim.columns = [c.lstrip() for c in confirmedCasesUSTrim.columns]
confirmedCasesUSTrim = confirmedCasesUSTrim.reindex(
    sorted(confirmedCasesUSTrim.columns), axis=1)


In [ ]:
confirmedDeathsTrim = confirmedDeaths.iloc[2:]
confirmedDeathsTrim.index = pd.to_datetime(confirmedDeathsTrim.index)
confirmedDeathsTrim = utils.computeGlobal(confirmedDeathsTrim)
confirmedDeathsTrim = utils.computeCasesOutside(confirmedDeathsTrim, 
                                               ['Mainland China', '!Global'],
                                               '!Outside Mainland China'
                                              )

In [ ]:
confirmedRecoveredTrim = confirmedRecovered.iloc[2:]
confirmedRecoveredTrim.index = pd.to_datetime(confirmedRecoveredTrim.index)
confirmedRecoveredTrim = utils.computeGlobal(confirmedRecoveredTrim)
confirmedRecoveredTrim = utils.computeCasesOutside(confirmedRecoveredTrim, 
                                               ['Mainland China', '!Global'],
                                               '!Outside Mainland China'
                                              )

In [ ]:
unknownOutcomes = confirmedCasesTrim - confirmedDeathsTrim - confirmedRecoveredTrim

In [ ]:
mortalityRate = confirmedDeathsTrim/confirmedCasesTrim

In [ ]:
recoveryRate = confirmedRecoveredTrim/confirmedCasesTrim

In [ ]:
knownOutcomeRate = (confirmedCasesTrim - unknownOutcomes)/confirmedCasesTrim

# Interactive plots

In [ ]:
import visualize
from ipywidgets import widgets
from ipywidgets import interact
from ipywidgets import fixed

In [ ]:
countries = list(confirmedCasesTrim.columns)
multiCountry = widgets.SelectMultiple(
    options=countries,
    value=['!Global', '!Outside Mainland China'],
    description='Country',
    disabled=False
)
log = widgets.Checkbox(value=False, description='Log scale')

In [ ]:
statesUS = list(confirmedCasesUSTrim.columns)
multiState = widgets.SelectMultiple(
    options=statesUS,
    value=['!Total US'],
    description='State',
    disabled=False
)

## Confirmed cases

In [ ]:
interact(visualize.plotTimeSeriesInteractive, 
         df=fixed(confirmedCasesTrim), 
         selectedColumns=multiCountry, 
         log=log,
         yLabel=fixed('Total confirmed cases'),
         title=fixed('COVID-19 total confirmed cases')
);

In [ ]:
interact(visualize.plotTimeSeriesInteractive, 
         df=fixed(confirmedCasesUSTrim), 
         selectedColumns=multiState, 
         log=log,
         yLabel=fixed('Total confirmed cases'),
         title=fixed('COVID-19 total confirmed cases in US states')
);

In [ ]:
def viewTopStates(n):
    print(confirmedCasesUSTrim.sum(axis=0).sort_values(ascending=False).head(n))

In [ ]:
interact(viewTopStates, n=widgets.IntSlider(min=1, max=len(statesUS), step=1, value=5));

In [ ]:
interact(visualize.plotTimeSeriesInteractive, 
         df=fixed(confirmedCasesTrim.diff(axis=0)), 
         selectedColumns=multiCountry, 
         log=log,
         yLabel=fixed('Confirmed cases (per day)'),
         title=fixed('COVID-19 confirmed new cases (per day)')
);

## Confirmed deaths

In [ ]:
interact(visualize.plotTimeSeriesInteractive, 
         df=fixed(confirmedDeathsTrim), 
         selectedColumns=multiCountry, 
         log=log,
         yLabel=fixed('Number of deaths'),
         title=fixed('COVID-19 total confirmed deaths')
);

## Untracked cases

In [ ]:
interact(visualize.plotTimeSeriesInteractive, 
         df=fixed(unknownOutcomes), 
         selectedColumns=multiCountry, 
         log=log,
         yLabel=fixed('Total unknown outcomes'),
         title=fixed('COVID-19 unknown outcomes from confirmed cases')
);

## Plot mortality rate

In [ ]:
interact(visualize.plotTimeSeriesInteractive, 
         df=fixed(mortalityRate), 
         selectedColumns=multiCountry, 
         log=log,
         yLabel=fixed('Mortality rate (deaths/cases)'),
         title=fixed('COVID-19 mortality rate')
);

## Plot recovery rate

In [ ]:
interact(visualize.plotTimeSeriesInteractive, 
         df=fixed(recoveryRate), 
         selectedColumns=multiCountry, 
         log=log,
         yLabel=fixed('Recovery rate (recovered/cases)'),
         title=fixed('COVID-19 recovery rate')
);

## Plot reporting rate

In [ ]:
interact(visualize.plotTimeSeriesInteractive, 
         df=fixed(knownOutcomeRate), 
         selectedColumns=multiCountry, 
         log=log,
         yLabel=fixed('Known outcome rate'),
         title=fixed('COVID-19 known outcome rate')
);